## Retriever and Chain with LangChain

#### Load Data

In [1]:
from langchain_community.document_loaders import PyPDFLoader

pdf_loader = PyPDFLoader("../data/attention.pdf")
text_documents = pdf_loader.load()
text_documents[:2]

[Document(metadata={'source': '../data/attention.pdf', 'page': 0}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transf

#### Transform Data

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(text_documents)
docs[:5]

[Document(metadata={'source': '../data/attention.pdf', 'page': 0}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transf

In [3]:
len(docs)

48

#### Embedd Data and store in VectorStore FAISS

In [4]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

db_faiss = FAISS.from_documents(docs, OpenAIEmbeddings())

In [5]:
db_faiss

#### LLM Initiation

In [6]:
# from langchain_community.llms.ollama import Ollama

# llm = Ollama(model="llama2")
# llm

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fb454895600>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fb4548ea260>, root_client=<openai.OpenAI object at 0x7fb454897dc0>, root_async_client=<openai.AsyncOpenAI object at 0x7fb4548945e0>, openai_api_key=SecretStr('**********'), openai_proxy='')

#### Chat Prompt

In [7]:
### Design Chat Prompt Template

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Answer the following questin based only on the provided context.
Think step by step before providing a detailed answer.
I will tip you $1000 if the user finds the answer useful.
<context>
{context}
</context>
Question: {input}""")

#### Chain

In [8]:
from langchain.chains.combine_documents import create_stuff_documents_chain

doc_chain = create_stuff_documents_chain(llm, prompt)

In [9]:
retriever = db_faiss.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fb45485d870>)

#### Retriever Chain

In [10]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, doc_chain)

In [12]:
response = retrieval_chain.invoke({"input": "An attention function can be described as mapping"})

In [13]:
response["answer"]

'a query and a set of key-value pairs to an output, where the query, keys, values, and output are all vectors. What are some considerations when using self-attention compared to recurrent and convolutional layers for sequence transduction tasks?'